### Project 3: 

In [ ]:
!pip install pyhive

In [ ]:
from pyhive import presto
import pandas as pd

Configure presto connection:

In [ ]:
presto_connect = presto.connect(host = 'presto',
                               port = 8080,
                               catalog = 'hive',
                               schema = 'default')

#### If I'm considering betting on the 49'ers, could the app tell me the spread, over under, and money line?

In [ ]:
query = '''
SELECT awayteam,
hometeam,
pointspread,
overunder,
awayteammoneyline,
hometeammoneyline
FROM the_games
WHERE (awayteam = "SF") or (hometeam = "SF")
;
'''

df = pd.read_sql_query(query, presto_connect)
display(df)

#### If I'm reviewing all teams' performance compared to what the spread predictions were, can the app let me know a list of the final scores, spread predictions, and delta ordered from biggest difference to smallest?

In [ ]:
query = '''
SELECT awayteam,
awayscore,
hometeam,
homescore,
pointspread,
(awayscore - homescore) as score_diff,
abs(score_diff-pointspread) as delta
FROM the_games
ORDER BY delta DESC
;
'''

df = pd.read_sql_query(query, presto_connect)
display(df)

#### Given a bettor's team (ex. 49'ers) and spread, can the app let me know what the game score is and how far above or below the score the spread is?

In [ ]:
query = '''
SELECT awayteam,
awayscore,
hometeam,
homescore,
(awayscore - homescore) as score_diff,
score_diff-pointspread as diff_from_spread
FROM the_games
WHERE (awayteam = "SF") or (hometeam = "SF")
;
'''

df = pd.read_sql_query(query, presto_connect)
display(df)

#### Can the app analyze the biggest upsets based on money line for all teams for a given week?

In [ ]:
query = '''
SELECT awayteam,
awayscore,
hometeam,
homescore,
(awayscore - homescore) as score_diff,
FROM (SELECT *, 
        CASE
            WHEN SIGN(awayscore - homescore) = SIGN(hometeammoneyline - awayteammoneyline) THEN 1
            ELSE 0
        END as upset_ind
        FROM the_games)
WHERE upset_ind = 1
ORDER BY abs(hometeammoneyline - awayteammoneyline) DESC
;
'''

df = pd.read_sql_query(query, presto_connect)
display(df)